<a href="https://colab.research.google.com/github/amrios80/Endeporte/blob/main/endeporte_proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENDEPORTE_PROYECTO_FINAL

### Objetivo:
Mediante tecnicas de machine learning establecer las variables que mayor relacion tienen con la desercion e identificar los estudiantes de carreras profesionales o tecnologicas de la entidad que pueden desertar o no de sus programas.

### Contexto del negocio
Para la entidad es dificil identificar tempranamente los estudiantes que están en riesgo de abandonar sus carreras, las intervenciones son a menudo menos efectivas porque las señales de riesgo no aparecen hasta que el estudiante ya ha tomado la decisión de desertar. Se espera que mediante modelos de machine learning se identifiquen con anticipacion los estudiantes que pueden llegar a desertar con el fin de brindar a cada estudiante el tipo y nivel de apoyo que necesita para superar sus dificultades.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)


##**1. Entendimiento y preparación de los datos:**


### Cargue del dataset

In [34]:
demograficos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/Demograficos.csv.gz"
historicos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/HistoricoNotas.csv.gz"

demograficos_df = pd.read_csv(demograficos_url, sep=";", compression='gzip')
historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")

<ipython-input-34-89934b14d202>:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")


### Limpieza de datos

* ***Estandarizacion*** : Segun las reglas acordadas con ENDEPORTE, Se excluyen del estudio las ESPECIALIZACIONES:

In [35]:
demograficos_df.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
18178,2019 - 2,53212182011,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,3,4,Masculino,22-05-1999,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,01/08/19,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2019-II,20,Edad entre 18 y 25 años,Fac. Deporte,Universitrario
21319,2019 - 2P,7802192P049,ESCUELA NACIONAL DEL DEPORTE,7802,ESPECIALIZACION EN DIRECCION Y GESTION DEPORTIVA,1,"4,2",Masculino,11-05-1990,2.0,PEREIRA,RISARALDA,Soltero,"4,2",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,23/08/19,27001.0,QUIBDO,27.0,CHOCO,0.0,NaN,1,2019-II,29,Edad entre 26 y 33 años,Fac. Admon,Especialización
43703,2023 - 1,53212222186,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,2,"3,3",Masculino,02-07-2004,1.0,CALI,VALLE DEL CAUCA,Soltero,"3,4",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,23/12/22,76001.0,CALI,76.0,VALLE DEL CAUCA,1.0,1.0,1,2023-I,18,Edad entre 18 y 25 años,Fac. Deporte,Universitrario


In [36]:
demograficos_df["NIVEL_FORMACION"].value_counts()

,count
NIVEL_FORMACION,
Universitrario,48022
Tecnológico,4994
Especialización,2403


In [37]:
demograficos_df=demograficos_df[(demograficos_df["NIVEL_FORMACION"] != "Especialización")]



*  ***Eliminacion de duplicados*** : La informacion demografica debe estar a nivel de estudiante, segun lo definido con la entidad, se valida que no haya duplicidad a nivel de codigo del estudiante.



In [56]:
total_rows = demograficos_df.shape[0]
total_rows

53016

In [40]:
demograficos_df.loc[demograficos_df.duplicated(subset=["CODIGO"], keep=False)].sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
9604,2018 - 1,53212131085,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,7,"4,2",Femenino,20-09-1994,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,7",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,15/01/18,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2018-I,24,Edad entre 18 y 25 años,Fac. Deporte,Universitrario
13711,2018 - 2,53212172368,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,3,"3,8",Femenino,25-08-1999,1.0,PRADERA,VALLE DEL CAUCA,Soltero,"3,9",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,04/09/18,76563.0,PRADERA,76.0,VALLE DEL CAUCA,0.0,NaN,1,2018-II,19,Edad entre 18 y 25 años,Fac. Deporte,Universitrario
34965,2021 - 2,90555191076,ESCUELA NACIONAL DEL DEPORTE,90555,ADMINISTRACION DE EMPRESAS,6,"3,7",Femenino,15-07-2001,1.0,CALI,VALLE DEL CAUCA,Soltero,4,EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,09/08/21,76001.0,CALI,76.0,VALLE DEL CAUCA,1.0,1.0,1,2021-II,20,Edad entre 18 y 25 años,Fac. Admon,Universitrario


In [51]:
demograficos_df[demograficos_df["CODIGO"]=="53212172368"].sort_values(by="PERIODO_ACAENICO",ascending=False).head(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
43077,2022 - 2,53212172368,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,7,"4,9",Femenino,25-08-1999,1.0,PRADERA,VALLE DEL CAUCA,Soltero,"4,3",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,29/06/22,76563.0,PRADERA,76.0,VALLE DEL CAUCA,0.0,NaN,1,2022-II,23,Edad entre 18 y 25 años,Fac. Deporte,Universitrario
39299,2022 - 1,53212172368,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,"4,6",Femenino,25-08-1999,1.0,PRADERA,VALLE DEL CAUCA,Soltero,"4,2",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,12/01/22,76563.0,PRADERA,76.0,VALLE DEL CAUCA,0.0,NaN,1,2022-I,23,Edad entre 18 y 25 años,Fac. Deporte,Universitrario
35461,2021 - 2,53212172368,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,6,"4,7",Femenino,25-08-1999,1.0,PRADERA,VALLE DEL CAUCA,Soltero,"4,2",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,12/07/21,76563.0,PRADERA,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,22,Edad entre 18 y 25 años,Fac. Deporte,Universitrario


Se observa que el codigo de estudiante se repite tantas veces como periodos académicos haya cursado y las variables de semestre, promediosemestre, promediogeneral y fecha_matricula son las correspondientes al periodo academico, las demas variables demograficas son las mismas para todos los periodos academicos. Por lo anterior, se tomara la informacion demografica correspondiente al ultimo periodo academico.

In [48]:
idx = demograficos_df.groupby('CODIGO')['PERIODO_ACAENICO'].idxmax()

In [49]:
max_periodo_rows = demograficos_df.loc[idx]

In [53]:
demograficos_df_uniq=max_periodo_rows.copy()

In [58]:
total_rows_uniq = demograficos_df_uniq.shape[0]
total_rows_uniq

10021

El estudio se realizara con la informacion demografica correspondiente al ultimo periodo academico de 10.021 estudiantes unicos.

***Formato de variables numericas y fechas:*** Creacion de la variable YEARNACIMIENTO, conversion a valor numerico del promedio general y de semestre.

In [59]:
demograficos_df_uniq["YEARNACIMIENTO"]=pd.to_datetime(demograficos_df_uniq["FECHANACIMIENTO"])
demograficos_df_uniq['YEARNACIMIENTO'] = demograficos_df_uniq['YEARNACIMIENTO'].dt.year
field="PROMEDIOGENERAL"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field].str.replace(',', '.'), errors='coerce')
field="PROMEDIOSEMESTRE"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field].str.replace(',', '.'), errors='coerce')

<ipython-input-59-0d1c1fa71a41>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  demograficos_df_uniq["YEARNACIMIENTO"]=pd.to_datetime(demograficos_df_uniq["FECHANACIMIENTO"])


In [61]:
demograficos_df_uniq.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION,YEARNACIMIENTO
53619,2024 - 1,104024211050,ESCUELA NACIONAL DEL DEPORTE,104024,TERAPIA OCUPACIONAL,7,4.6,Femenino,19-08-2000,2.0,BUENAVENTURA,VALLE DEL CAUCA,Soltero,4.3,ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,MIOPIA,DISCAPACIDAD SENSORIAL - BAJA VISIÓN,03/01/24,76109.0,BUENAVENTURA,76.0,VALLE DEL CAUCA,0.0,NaN,1,2024-I,24,Edad entre 18 y 25 años,Fac. Salud,Universitrario,2000
32434,2021 - 2,7306202199,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,3,2.4,Femenino,10-03-2002,3.0,CALI,VALLE DEL CAUCA,Soltero,3.4,BAJO RENDIMIENTO ACADEMICO,ANTIGUO,BAJO RENDIMIENTO ACADEMICO,ANTIGUO,NINGUNA,NINGUNA,16/07/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,19,Edad entre 18 y 25 años,Fac. Salud,Universitrario,2002
51869,2024 - 1,90555201022,ESCUELA NACIONAL DEL DEPORTE,90555,ADMINISTRACION DE EMPRESAS,9,4.2,Masculino,23-12-2001,2.0,CALI,VALLE DEL CAUCA,Soltero,4.2,ACTIVO,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,RUPTURA EN LA CLAVÍCULA,DISCAPACIDAD FISICA O MOTORA,08/01/24,76109.0,BUENAVENTURA,76.0,VALLE DEL CAUCA,1.0,3.0,1,2024-I,23,Edad entre 18 y 25 años,Fac. Admon,Universitrario,2001


## Limpieza de datos
### imputacion de datos faltantes
### estandarizacion

### creacion de nuevas caracteristicas
### codificacion de variables categoricas
### normalizacion

